# Dataframe basics for PySpark

In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import pandas as pd
sc = spark.sparkContext

## Make a sample dataframe from Titanic data

In [2]:
data1 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William'},
         'Sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}}

data2 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Age': {0: 22, 1: 38, 2: 26, 3: 35, 4: 35},
         'Fare': {0: 7.3, 1: 71.3, 2: 7.9, 3: 53.1, 4: 8.0},
         'Pclass': {0: 3, 1: 1, 2: 3, 3: 1, 4: 3}}

df1_pd = pd.DataFrame(data1, columns=data1.keys())
df2_pd = pd.DataFrame(data2, columns=data2.keys())

In [3]:
df1_pd

,PassengerId,Name,Sex,Survived
0,1,Owen,male,0
1,2,Florence,female,1
2,3,Laina,female,1
3,4,Lily,female,1
4,5,William,male,0


In [4]:
df2_pd

,PassengerId,Age,Fare,Pclass
0,1,22,7.3,3
1,2,38,71.3,1
2,3,26,7.9,3
3,4,35,53.1,1
4,5,35,8.0,3


### Convert pandas dataframe to Spark dataframe

In [5]:
df1 = spark.createDataFrame(df1_pd)
df2 = spark.createDataFrame(df2_pd)
df1.show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



In [6]:
df1.printSchema()

root
 |-- PassengerId: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Survived: long (nullable = true)



### Basic dataframe verbs

Here are the 5 basic verbs for Spark, reproducing the basic verbs in R's dplyr package

#### Select
Takes either a list of column names or an unpacked list.

In [7]:
cols1 = ['PassengerId', 'Name']
df1.select(cols1).show()

+-----------+--------+
|PassengerId|    Name|
+-----------+--------+
|          1|    Owen|
|          2|Florence|
|          3|   Laina|
|          4|    Lily|
|          5| William|
+-----------+--------+



#### Filter
Filter takes column expression or SQL expression ---
- Filter with column expression

In [8]:
df1.filter(df1.Sex == 'female').show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
+-----------+--------+------+--------+



- Filter with SQL expression. Note the double and single quotes

In [9]:
df1.filter("Sex='female'").show() 

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
+-----------+--------+------+--------+



#### Mutate, or creating new columns
Creating new columns in Spark uses `.withColumn()`. I have yet found a convenient way to create multiple columns at once without chaining multiple `.withColumn()` methods.

In [10]:
df2.withColumn('AgeTimesFare', df2.Age*df2.Fare).show()

+-----------+---+----+------+------------+
|PassengerId|Age|Fare|Pclass|AgeTimesFare|
+-----------+---+----+------+------------+
|          1| 22| 7.3|     3|       160.6|
|          2| 38|71.3|     1|      2709.4|
|          3| 26| 7.9|     3|       205.4|
|          4| 35|53.1|     1|      1858.5|
|          5| 35| 8.0|     3|       280.0|
+-----------+---+----+------+------------+



#### Summarize and group by
To summarize or aggregate a dataframe, first I need to convert the dataframe to a `GroupedData` object with `groupby()`, then call the aggregate functions. 

In [11]:
gdf2 = df2.groupby('Pclass')
gdf2

I can take the average of columns by passing an _unpacked_ list of column names.

In [12]:
avg_cols = ['Age', 'Fare']
gdf2.avg(*avg_cols).show()

+------+------------------+-----------------+
|Pclass|          avg(Age)|        avg(Fare)|
+------+------------------+-----------------+
|     1|              36.5|             62.2|
|     3|27.666666666666668|7.733333333333333|
+------+------------------+-----------------+



To call multiple aggregation functions at once, pass a dictionary. 

In [13]:
gdf2.agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'}).show()

+------+--------+------------------+---------+
|Pclass|count(1)|          avg(Age)|sum(Fare)|
+------+--------+------------------+---------+
|     1|       2|              36.5|    124.4|
|     3|       3|27.666666666666668|     23.2|
+------+--------+------------------+---------+



To rename the columns `count(1)`, `avg(Age)` etc, use `toDF()`.

In [14]:
(
    gdf2
    .agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'})
    .toDF('Pclass', 'counts', 'average_age', 'total_fare')
    .show()
)

+------+------+------------------+----------+
|Pclass|counts|       average_age|total_fare|
+------+------+------------------+----------+
|     1|     2|              36.5|     124.4|
|     3|     3|27.666666666666668|      23.2|
+------+------+------------------+----------+



#### Arrange (sort)
Use the `.sort()` method to sort the dataframes. I haven't seen a good use case for this when I use Spark though. 

In [15]:
df2.sort('Fare', ascending=False).show()

+-----------+---+----+------+
|PassengerId|Age|Fare|Pclass|
+-----------+---+----+------+
|          2| 38|71.3|     1|
|          4| 35|53.1|     1|
|          5| 35| 8.0|     3|
|          3| 26| 7.9|     3|
|          1| 22| 7.3|     3|
+-----------+---+----+------+



### Joins and unions
There are two ways to combine dataframes --- joins and unions. The idea here is the same as joining and unioning tables in SQL. 

#### Joins

For example, I can join the two titanic dataframes by the column `PassengerId`

In [17]:
df1.join(df2, ['PassengerId']).sort('PassengerId').show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          1|    Owen|  male|       0| 22| 7.3|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          4|    Lily|female|       1| 35|53.1|     1|
|          5| William|  male|       0| 35| 8.0|     3|
+-----------+--------+------+--------+---+----+------+



I can also join by conditions, but it creates duplicate column names if the keys have the same name. I found this behavior frustrating. For now, the only way to avoid this is to pass a list of join keys as in the previous cell. If I want to make nonequi joins, then I need to rename the keys before I join.

#### Nonequi joins

Here is an example of nonequi join. They can be _very_ slow due to skewed data, but this is one operation that Spark can do while Hive can not.

In [18]:
df1.join(df2, df1.PassengerId <= df2.PassengerId).show() # Note the duplicate col names

+-----------+--------+------+--------+-----------+---+----+------+
|PassengerId|    Name|   Sex|Survived|PassengerId|Age|Fare|Pclass|
+-----------+--------+------+--------+-----------+---+----+------+
|          1|    Owen|  male|       0|          1| 22| 7.3|     3|
|          1|    Owen|  male|       0|          2| 38|71.3|     1|
|          1|    Owen|  male|       0|          3| 26| 7.9|     3|
|          1|    Owen|  male|       0|          4| 35|53.1|     1|
|          1|    Owen|  male|       0|          5| 35| 8.0|     3|
|          2|Florence|female|       1|          2| 38|71.3|     1|
|          2|Florence|female|       1|          3| 26| 7.9|     3|
|          2|Florence|female|       1|          4| 35|53.1|     1|
|          2|Florence|female|       1|          5| 35| 8.0|     3|
|          3|   Laina|female|       1|          3| 26| 7.9|     3|
|          3|   Laina|female|       1|          4| 35|53.1|     1|
|          3|   Laina|female|       1|          5| 35| 8.0|   

#### Unions

`Union()` returns a dataframe from the union of two dataframes

In [19]:
df1.union(df1).show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



Some of my iterative algorithms create chained `union()` objects. There is a potential catch that the execution plan may grow too long, which cause performance problems or errors. 

One common symptom of performance issues caused by chained unions in a for loop is it took longer and longer to iterate through the loop. In this case, `repartition()` and `checkpoint()` may help solving this problem.

## Dataframe input and output (I/O)

There are two classes `pyspark.sql.DataFrameReader` and `pyspark.sql.DataFrameWriter` that handles dataframe I/O. Depending on the configuration, the files may be saved locally, through a Hive metasore, or to a Hadoop file system (HDFS). 

Common methods on saving dataframes to files include `saveAsTable()` for Hive tables and `saveAsFile()` for local or Hadoop file system.

I will refer to the documentation on how to read and write dataframes. 

## The spark.sql API

Many of the operations that I showed can be accessed by writing SQL (Hive) queries in `spark.sql()`. This is also a convenient way to read Hive tables into Spark dataframes. To make an existing Spark dataframe usable for `spark.sql()`, I need to register said dataframe as a temporary table.

### Temp tables
As an example, I can register the two dataframes as temp tables then join them through `spark.sql()`.

In [20]:
df1.createOrReplaceTempView('df1_temp')
df2.createOrReplaceTempView('df2_temp')

In [22]:
query = '''
    select 
        a.PassengerId,
        a.Name,
        a.Sex,
        a.Survived,
        b.Age,
        b.Fare,
        b.Pclass
    from df1_temp a
    join df2_temp b
        on a.PassengerId = b.PassengerId'''
dfj = spark.sql(query)

In [23]:
dfj.show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          5| William|  male|       0| 35| 8.0|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          4|    Lily|female|       1| 35|53.1|     1|
+-----------+--------+------+--------+---+----+------+



## Going deeper

In this section, I will go through some idea and useful tools associated with said ideas that I found helpful in tuning performance or debugging dataframes. The first of which is the difference between two types of operations: _transformations_ and _actions_, and a method `explain()` that prints out the execution plan of a dataframe.

### Explain(), transformations, and actions

When I create a dataframe in PySpark, unlike Python objects, dataframes are lazy evaluated. What it means is that most operations are _transformations_ that modify the _execution plan_ on how Spark should handle the data, but the plan is not executed until we call and _action_.

For example, if I want to join `df1` and `df2` on the key `PassengerId` as before:

In [22]:
df1.explain()

== Physical Plan ==
Scan ExistingRDD[PassengerId#0L,Name#1,Sex#2,Survived#3L]


In [23]:
df2.explain()

== Physical Plan ==
Scan ExistingRDD[PassengerId#9L,Age#10L,Fare#11,Pclass#12L]


In [24]:
dfj1 = df1.join(df2, ['PassengerId'])
dfj1.explain()

== Physical Plan ==
*Project [PassengerId#0L, Name#1, Sex#2, Survived#3L, Age#10L, Fare#11, Pclass#12L]
+- *SortMergeJoin [PassengerId#0L], [PassengerId#9L], Inner
   :- *Sort [PassengerId#0L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(PassengerId#0L, 200)
   :     +- *Filter isnotnull(PassengerId#0L)
   :        +- Scan ExistingRDD[PassengerId#0L,Name#1,Sex#2,Survived#3L]
   +- *Sort [PassengerId#9L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(PassengerId#9L, 200)
         +- *Filter isnotnull(PassengerId#9L)
            +- Scan ExistingRDD[PassengerId#9L,Age#10L,Fare#11,Pclass#12L]


In this case, `join()` is a transformation that laid out a plan for Spark to join the two dataframes, but it wasn't executed unless I call an action, such as `.count()`, that has to go through the actual data defined by `df1` and `df2` in order to return a Python object (integer).

In [25]:
dfj1.count()

5

What if I join through the `spark.sql()` API instead of calling `.join()`? In this case, both happen to generate the same execution plan. But it's not always like this. A good execution plan equals good performance, and I `explain()` a lot when I need to tune performance of Spark jobs.

In [26]:
query = '''
    select 
        a.PassengerId,
        a.Name,
        a.Sex,
        a.Survived,
        b.Age,
        b.Fare,
        b.Pclass
    from df1_temp a
    join df2_temp b
        on a.PassengerId = b.PassengerId'''
dfj = spark.sql(query)
dfj.explain()

== Physical Plan ==
*Project [PassengerId#0L, Name#1, Sex#2, Survived#3L, Age#10L, Fare#11, Pclass#12L]
+- *SortMergeJoin [PassengerId#0L], [PassengerId#9L], Inner
   :- *Sort [PassengerId#0L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(PassengerId#0L, 200)
   :     +- *Filter isnotnull(PassengerId#0L)
   :        +- Scan ExistingRDD[PassengerId#0L,Name#1,Sex#2,Survived#3L]
   +- *Sort [PassengerId#9L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(PassengerId#9L, 200)
         +- *Filter isnotnull(PassengerId#9L)
            +- Scan ExistingRDD[PassengerId#9L,Age#10L,Fare#11,Pclass#12L]


### Data persistence: cache() and checkpoint()

#### caching

Proper caching is the key to high performance Spark. I, however, still don't have good intuition on when to cache and when not to cache. 

A rule of thumb is that 

* **Cache a dataframe when it is used multiple times in the script.**

Keep in mind that it is only cached _after the first action_, such as `saveAsTable()`. If for whatever reason I want to make sure the data is cached before I save the dataframe, then I have to call an action like `.count()`.

In [27]:
df1.cache()

DataFrame[PassengerId: bigint, Name: string, Sex: string, Survived: bigint]

This also works as `.cache()` is an inplace method.

In [28]:
df1 = df1.cache()

To check if a dataframe is cached, check the `storageLevel` property.

In [29]:
df1.storageLevel

StorageLevel(True, True, False, True, 1)

To un-cache a dataframe, use `unpersist()`

In [30]:
df1.unpersist()
df1.storageLevel

StorageLevel(False, False, False, False, 1)

There are 4 caching levels that can be fine-tuned with `persist()`, but I have not seen a use case where fine tuning has been worthwhile. Refer to the `persist()` documentation for detail

#### Checkpointing

I said that sometimes chaining too many `union()` cause performance problem or even out of memory errors. `checkpoint()` truncates the execution plan and saves the checkpointed dataframe to a temporary location on the disk.

[Jacek Laskowski recommends caching before checkpointing](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-rdd-checkpointing.html) so Spark doesn't have to read in the dataframe from disk after it's checkpointed.

To use `checkpoint()`, I need to specify the temporary file location to save the datafame to by accessing the `sparkContext` object from `SparkSession`.

In [31]:
sc = spark.sparkContext
sc.setCheckpointDir("/checkpointdir") # save to c:/checkpointdir

For example, I can join `df1` to itself 3 times:

In [32]:
df = df1.join(df1, ['PassengerId'])
df.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*Project [PassengerId#0L, Name#1, Sex#2, Survived#3L, Name#287, Sex#288, Survived#289L, Name#299, Sex#300, Survived#301L]
+- *SortMergeJoin [PassengerId#0L], [PassengerId#298L], Inner
   :- *Project [PassengerId#0L, Name#1, Sex#2, Survived#3L, Name#287, Sex#288, Survived#289L]
   :  +- *SortMergeJoin [PassengerId#0L], [PassengerId#286L], Inner
   :     :- *Sort [PassengerId#0L ASC NULLS FIRST], false, 0
   :     :  +- Exchange hashpartitioning(PassengerId#0L, 200)
   :     :     +- *Filter isnotnull(PassengerId#0L)
   :     :        +- Scan ExistingRDD[PassengerId#0L,Name#1,Sex#2,Survived#3L]
   :     +- *Sort [PassengerId#286L ASC NULLS FIRST], false, 0
   :        +- Exchange hashpartitioning(PassengerId#286L, 200)
   :           +- *Filter isnotnull(PassengerId#286L)
   :              +- Scan ExistingRDD[PassengerId#286L,Name#287,Sex#288,Survived#289L]
   +- *Sort [PassengerId#298L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(PassengerId#298L, 20

I can also `checkpoint()` after the first join to truncate the plan.

In [33]:
df = df1.join(df1, ['PassengerId']).checkpoint()
df.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*Project [PassengerId#0L, Name#1, Sex#2, Survived#3L, Name#314, Sex#315, Survived#316L, Name#335, Sex#336, Survived#337L]
+- *SortMergeJoin [PassengerId#0L], [PassengerId#334L], Inner
   :- *Filter isnotnull(PassengerId#0L)
   :  +- Scan ExistingRDD[PassengerId#0L,Name#1,Sex#2,Survived#3L,Name#314,Sex#315,Survived#316L]
   +- *Sort [PassengerId#334L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(PassengerId#334L, 200)
         +- *Filter isnotnull(PassengerId#334L)
            +- Scan ExistingRDD[PassengerId#334L,Name#335,Sex#336,Survived#337L]


### Partitions and repartition()

Another common cause of performance problems for me was having too many partitions. To check the number of partitions, use `.rdd.getNumPartitions()`

In [34]:
df1.rdd.getNumPartitions()

4

This dataframe, despite having only 5 rows, has 4 partitions. This is too many. I can repartition to only 1 partition.

In [35]:
df1_repartitioned = df1.repartition(1)
df1_repartitioned.rdd.getNumPartitions()

1